# レポート課題

以下の問題について，この `report.ipynb` を編集し，完成した自分が編集した .ipynb ファイルをGoogle Classroomの該当課題に提出してください．（もしこのノートブックをファイルに保存したいときには，メニューの「file」→「Download」で .ipynb ファイルをダウンロード出来るので，それを提出してください．）


- 適宜コメントを入れてください．
- 提出する際のファイル名は，「`学籍番号.ipynb`」としてください．
- 問題1について，Markdownで書きにくい場合には，ClassroomにPDFファイルを追加して提出してもかまいません．
- 発展課題は任意とします．
- Markdownを書くときには，[Wiki](https://ja.wikipedia.org/wiki/Markdown)にあるぐらいの基本的なマークアップで書いてくれればよいです（もちろん，自分で調べてもっと高度なMarkdownを書いてくれても見られればよいです）．

# 問題1 - LALR構文解析についてまとめる（第2回の内容から）

Yaccは，***LALR構文解析*** を行う構文解析器を出力するということを第2回で説明しました．

LALR構文解析について，以下の用語などを交えて説明してください．
（すべてが含まれている必要はないですが，自分でちゃんと理解したうえでまとめること）

- 上向き構文解析
- スタック
- 構文解析表
- Shift / Reduce
- shift/reduce conflict の例
- reduce/reduce conflictの例
- 第1回の問題5 （右再帰と左再帰で，どちらがYaccにとって良いか．）

ヒント1: 書籍が沢山出ているので，それを参考にしてください．（僕らのときは，）

> 『情報科学こんせぷつ8 コンパイラの仕組み』渡邊 坦(著) ISBN：978-4-254-12708-9

が教科書に指定されていました．

ヒント2: 今回のノートブックが裏で使っているBisonのページは次のとおり，
[The Bison Parser Algorithm](https://www.gnu.org/software/bison/manual/html_node/Algorithm.html)

### 1. 解析アルゴリズムの位置づけ

LALR構文解析には、下記の特徴がある。

- **ボトムアップ方式**：入力（左→右）を読みながら右端導出を逆順にたどり、還元（*reduce*）を繰り返して開始記号へ到達する。
- **LR ファミリ**：LR(0) → SLR(1) → ***LALR(1)*** → Canonical LR(1) の順に構文解析表のサイズと解析力のトレードオフがある。LALR(1) は LR(1) と同じ先読み 1 トークンの解析力をほぼ維持しつつ、マージによって構文解析表が LR(0) とほぼ同じ大きさになるため実用パーサ―生成器（Yacc/Bison など）の標準方式となっている。
- まず LR(0) と同様の処理：LR(0) アイテム集合族（DFA）と GOTO 遷移を構成し、構文解析表の骨格を得る。
- その後，全ての閉包に対して先読みを付加：各 LR(0) 状態（閉包）に含まれるすべてのアイテムに，文脈に応じた先読み記号を付与して LR(1) アイテムへ拡張する。
- 初期状態から順に，全閉包の項を LR(1) 項に変換：DFA を BFS 順にたどりながら拡張を行うことで，解析表全体に一貫した先読み情報を行き渡らせる。
- 先読みの計算には Follow 集合を用いる：還元が妥当であるかを判断する先読み集合（look‑ahead set）は，Follow 集合によって導出される。
- LR(1) のように状態ごとに完全な lookahead を保持するのではなく、LR(0) 状態で lookahead を共有（マージ）することで、LR(1) より小さな表サイズを実現する（DeRemer & Pennello, 1982）。

### 1.1 Follow 集合とは？

**Follow(A)** とは，文法の開始記号から導出される sentential form において，非終端記号 **A** の直後に現れ得る *終端記号* の集合である。
具体的には，次を満たす終端記号 **a** は **Follow(A)** に含まれる。

1. 開始記号 **S** からの導出で `S ⇒* α A a β` が成り立つ。
2. ある規則 `B → α A` があり，`α` を消去できる（ε 推移）場合には，**Follow(B)** も **Follow(A)** に入る（再帰的定義）。
3. 文末記号 `$` は，開始記号が `S → α A` の形で終わり得るとき **Follow(A)** に含まれる。

#### 役割

- **SLR/LALR/LR(1) 解析**での *reduce* 動作判定：規則 `A → β` の還元が適用されるのは，現在の先読みトークンが **Follow(A)** に含まれるときに限る。
- **Look‑ahead 集合の種集合**：DeRemer–Pennello の LALR(1) look‑ahead 計算では，非終端遷移に関する Read/Follow 関係をグラフとして構築し，SCC の伝播により LA 集合を求める。その際，Follow 集合が初期値として用いられる。

#### 算出アルゴリズム 

1. すべての非終端記号 **A** について **Follow(A) ← ∅** とする。
2. **Follow(S) ← { $ }** （`S` は開始記号）。
3. 規則を走査し，以下を満たす限り集合を更新する：
   - 規則 `B → α A a β` があり **a** が終端記号なら **a** を **Follow(A)** に追加。
   - 規則 `B → α A β` があり `β ⇒* ε` となる場合，**Follow(B)** を **Follow(A)** に併合。
4. 変化がなくなるまで 3 を繰り返す（固定点反復）。
---

### 2. 構文解析表

LALR パーサーは 2 枚のテーブルと 1 本のスタックから構成される。

| テーブル       | 行  | 列     | 値                                               | 役割      |
| ---------- | -- | ----- | ----------------------------------------------- | ------- |
| **ACTION** | 状態 | 終端記号  | `shift s_j` / `reduce A→β` / `accept` / `error` | 次の動作を決定 |
| **GOTO**   | 状態 | 非終端記号 | 次状態                                             | 還元後の遷移先 |

例えば、下記のようなACTION表が考えられます。この時、siはshift状態iをスタックに積む動作を、rjは文法jによるreduce動作を、accは受理を意味します。

| state | id | +  | \* | (  | )  | \$  |
| ----- | -- | -- | -- | -- | -- | --- |
| 0     | s5 |    |    | s4 |    |     |
| 1     |    | s6 |    |    |    | acc |
| 2     |    | r2 | s7 |    | r2 | r2  |

例えば下記のようなGOTO表が考えられます

| state | E | T | F |
| ----- | - | - | - |
| 0     | 1 | 2 | 3 |
| 4     |   | 8 | 3 |
| 6     |   |   | 9 |

#### スタック構造

`[ s₀, X₁, s₁, X₂, s₂, … ]` のように **状態と文法記号**を交互に積む。動作規則は次の通り。

1. **shift**: `ACTION[s_m, a] = shift s_j` のとき `(a, s_j)` をプッシュ。
2. **reduce**: `ACTION[s_m, a] = reduce A→β` のとき `2·|β|` 個ポップし `(A, GOTO[s_i, A])` をプッシュ。
3. **accept**: `ACTION[s_m, $] = accept` で解析終了。

SHIFT/REDUCE の衝突（*conflict*）が解決できない場合、文法はその方式で解析不可能（あるいは曖昧）となる。

> #### reduce/reduce conflict 例
>
> ```bnf
> sequence       : /* ε */
>               | maybeword
>               | sequence word
> ;
> maybeword      : /* ε */
>               | word
> ;
> ```
>
> `word` を読んだとき `maybeword → ε` で reduce するか、`word` を shift して `maybeword → word` を待つかが両立しない。

---

### 3. **LR(1) ➜ LALR(1) 解析表マージ手順**

1. **Canonical LR(1) 項集合族を生成**
   - 各項は形 `A→α·β , a` （先読み a を伴う）。
2. **コア抽出**
   - 先読みを除いた部分 `A→α·β` が **LR(0) コア**。
3. **同一コアをもつ状態同士をマージ**
   - 先読み集合を集合和で統合する。
4. **ACTION/GOTO 表を再構築**
   - マージ後の状態番号でエントリを張り直す。
5. **衝突検査**
   - マージにより新たな *reduce/reduce* または *shift/reduce* conflict が生じないか確認する。

 **ポイント**

- マージにより表サイズは LR(1) の 10⁻³ 倍程度まで縮小する例もある。
- 解析力は LR(1) にほぼ匹敵（ただしコンフリクトが新たに増える可能性がある）。

---

### 4. shift / reduce / accept 動作（詳細）

#### 4.1 shift

```
… sᵢ  ⇒  … sᵢ  a  sⱼ
        ACTION[sᵢ, a] = shift sⱼ
```

- トークン `a` を読み、スタックに `(a, sⱼ)` をプッシュ。

#### 4.2 reduce A→β

```
… sᵢ β  ⇒  … sᵢ A sₖ
          sₖ = GOTO[sᵢ, A]
```

- `2·|β|` 個の要素をポップして非終端 `A` と新状態 `sₖ` をプッシュ。

#### 4.3 accept

```
ACTION[s_m, $] = accept
```

- 入力全体が文法の開始記号に還元されたことを示す。

---

### 5. shift/reduce conflict とは？

同一状態・同一先読みトークンに対し **shift** と **reduce** の両候補が存在する状況。Yacc/Bison は既定で *shift* を優先し、警告を出力するが、仕様を意図どおりに満たす保証はなくなる。

```yacc
%token NEWLINE SPECIFIER
%token EQUALS STRING VAR
%%
st      : toplevel ;

toplevel: /* 空行 */
        | toplevel VAR EQUALS STRING NEWLINE
        | toplevel list NEWLINE
        ;

list    : maybe_specifier VAR
        | list maybe_specifier VAR
        ;

maybe_specifier: /* ε */
               | SPECIFIER /* shift/reduce conflict の温床 */
               ;
%%
```

---

### 6. 参考資料

- 筑波大学 HPC 講義資料 [https://www.hpcs.cs.tsukuba.ac.jp/\~msato/lecture-note/comp-lecture/note4.html](https://www.hpcs.cs.tsukuba.ac.jp/~msato/lecture-note/comp-lecture/note4.html)
- 近畿大学 コンパイラ講義資料 [https://www.info.kindai.ac.jp/compiler/lecture/Compiler12note.pdf](https://www.info.kindai.ac.jp/compiler/lecture/Compiler12note.pdf)
- DeRemer, F., & Pennello, T. (1982). *Efficient Computation of LALR(1) Look-Ahead Sets*. University of California, Santa Cruz, and MetaWare™ Incorporated.[https://dl.acm.org/doi/pdf/10.1145/69622.357187](https://dl.acm.org/doi/pdf/10.1145/69622.357187)


# 問題2 - 電卓の拡張（第1回の内容から）

第1回で作成した電卓を自由に拡張して，以下にコードを載せてください．
どんな機能を追加したかをMarkdownにまとめて書いて，コードには適宜コメントを入れてください．

- 授業の練習問題で自分が作成したものを含んでいてもよいです．
- Yaccの**演算子順序**について自分で調べて利用してくれてもよいです．
- 授業で対応していなかった演算・実数・関数に対応するなど，一般の関数電卓にある機能でもよいですし，自分で考えた新たな機能でもよいです（プログラムになってしまうと，次の問題3になってしまうので，あくまで電卓の範囲で）

- 積と商の計算を正しい順で(優先して)行う
- 電卓内の値を全てdobuleで扱い、小数点を含む演算を可能にする
- 三角関数の演算(math.hを使えなかったので、テイラー展開をしてcos,sin,tanを求めました)
- ルート計算
- 実数を用いた計算
- 0で割った時のエラー処理

In [1]:
/* lex f2.lex */
%{
#include <math.h>
#include <ctype.h>
#include <stdlib.h>
%}

digit   [0-9]
white   [\t ]
dot     ["."]
sign    [+-]

%%
"cos"        { return COS; }
"sin"        { return SIN; }
"tan"        { return TAN; }
"sqrt"       { return SQRT; }

{sign}?{digit}+{dot}{digit}+  { yylval.db = atof(yytext); return RNUM; }
{sign}?{digit}+               { yylval.db = atof(yytext); return RNUM; }

[*%/+()]     { return yytext[0]; }
"\n"         { return '\n'; }
{white}      { ; }


[Lex] flex generates lex.yy.c successfully

In [2]:
/* c mytrig.h */
#ifndef MYTRIG_H
#define MYTRIG_H

/* 数値は double 前提。fabs の代わりに自作 abs_d を使う */
static inline double abs_d(double x) { return x < 0 ? -x : x; }

/* 円周率の定義 */
#define MY_PI 3.14159265358979323846264338327950288419716939937510

/*余りを求めるコード*/
static double my_mod(double x,double y){
    return (double)((int)x % (int)y);
}
/*平方根をニュートン法で計算するアルゴリズム*/
static double my_sqrt(double num) {
    double x = num;
    double dx;
    while (1) {
        dx = (x * x - num) / (2.0 * x);
        if (dx * dx < 0.00001) break;
        x -= dx;
    }
    return x;
}

/* 角度を [-pi, pi] に正規化 */
static double reduce_pi(double x) {
    double two_pi = 2.0 * MY_PI;
    while (x >  MY_PI) x -= two_pi;
    while (x < -MY_PI) x += two_pi;
    return x;
}

/* テイラー展開をある程度の次数で打ち切り */
static double my_sin(double x) {
    // x を [-pi, pi] に正規化
    x = reduce_pi(x);
    double term = x;    // x^(2n+1)
    double sum  = x;
    double x2   = x * x;
    term *= -x2 / (2.0 * 3.0);
    sum  += term;
    term *= -x2 / (4.0 * 5.0);
    sum  += term;
    term *= -x2 / (6.0 * 7.0);
    sum  += term;
    term *= -x2 / (8.0 * 9.0);
    sum  += term;
    return sum;
}
/* テイラー展開でcosを求める関数*/
static double my_cos(double x) {
    // x を [-pi, pi] に正規化
    x = reduce_pi(x);
    double term = 1.0;  // x^(2n)
    double sum  = 1.0;
    double x2   = x * x;

    term *= -x2 / (1.0 * 2.0);
    sum  += term;
    term *= -x2 / (3.0 * 4.0);
    sum  += term;
    term *= -x2 / (5.0 * 6.0);
    sum  += term;
    term *= -x2 / (7.0 * 8.0);
    sum  += term;

    return sum;
}
/* テイラー展開でtanを求める関数*/
static double my_tan(double x) {
    double c = my_cos(x);
    if (abs_d(c) < 1e-12) {
        return (x >= 0) ? 1e12 : -1e12;
    }
    return my_sin(x) / c;
}

#endif /* MYTRIG_H */


/usr/bin/ld: /usr/lib/gcc/x86_64-linux-gnu/12/../../../x86_64-linux-gnu/libl.a(libmain.o): in function `main':
(.text.startup+0x9): undefined reference to `yylex'
collect2: error: ld returned 1 exit status
[C] gcc exited with code 1

In [35]:
/* yacc f2.yacc */
%{
#include <stdio.h>
#include <stdlib.h>
#include "mytrig.h"
%}
%union {
    double db;
}
%token <db> RNUM COS SIN TAN SQRT
%type <db> expr term prim
%left '+'
%left '*' '/' '%'
%%
exprlist:
        | exprlist expr '\n'    { printf("%.2f\n", $2); }
        ;

expr    : term
        | expr '+' term         { $$ = $1 + $3; } /*足し算*/
        | expr '-' term         { $$ = $1 - $3; } /*引き算*/
        ;
term    : prim       
        | term '*' term {$$  = $1 * $3; } /*掛け算*/
        | term '/' term {
                if($3 == 0){
                    printf("Error: division by zero\n");
                    exit(1);
                }else{
                    $$  = $1 / $3; 
                }
        } /*割り算*/
        | term '%' term {
                if($3 == 0){
                    printf("Error: division by zero\n");
                    exit(0);
                }else{
                    $$ = my_mod($1,$3);
                }
        } /*余り演算*/
        ;
prim
    : RNUM                      { $$ = $1; }
    | '(' expr ')'              { $$ = $2; }
    | COS '(' expr ')'          { $$ = my_cos($3); }
    | SIN '(' expr ')'          { $$ = my_sin($3); }
    | TAN '(' expr ')'          { $$ = my_tan($3); }
    | SQRT '(' expr ')'         { $$ = my_sqrt($3); }
    ;


[Yacc] bison generates y.tab.c successfully

In [3]:
/* c f2.c */
#include <stdio.h>
extern char *yytext;

#include "y.tab.c"
#include "lex.yy.c"

int main() {
  yyparse();
  return 0;
}

[C] gcc generates a.out successfully

In [4]:
/* a.out */
cos(-1) + 4.5 + -3.21 + tan(3) + sqrt(6)

4.13


In [5]:
/* a.out */
1/0

Error: division by zero


# 問題3 - 自分のプログラミング言語の拡張（第2・3回の内容から）

第2・3回で作成したプログラミング言語を自由に拡張して，以下にコードを載せてください．どんな機能を追加したか（複数推奨）をMarkdownでまとめて書いて，コードには適宜コメントを入れてください．

- 授業の練習問題で自分が作成したものを含んでもよいです．
- インタプリタでもコンパイラでもどちらでもよいです．
  - コンパイラの場合，出力先の言語は授業でやったC・x64アセンブリ言語のどちらかにしてください（出力先を変えてみるのは，以下で任意発展課題としています）
- たとえば，以下のような拡張が考えられます．（もちろん，これ以外の拡張もどんどん入れてください）
  - `if`文
  - `for`文
  - `switch`文
  - サブルーチン（値を返さない）
  - 関数（値を返す）
    - アセンブリ言語でやるとしたら，ラベルなどを付けてジャンプすることになるはず．どうスタックを使うかなどを考えないといけない．
  - 配列（1次元でよい）

下記のような機能を実装しました。
- if文
- switch文
- 掛け算、割り算、あまりの計算

In [65]:
/* lex f3.lex */
alpha   [a-zA-Z]
digit   [0-9]
white   [\n\t ]
%%
read                        { return READ;    } /*文字入力*/
print                       { return PRINT;   } /*文字出力*/
while                       { return WHILE;   } /*while文*/
"if"                        { return IF;      } /*if文*/
"else"                      { return ELSE;    } /*else文*/
"case"                      { return CASE;    } /*case文*/
"default"                   { return DEFAULT; } /*default文*/
"switch"                    { return SWITCH;  } /*switch文*/
"++"                        { return PLUSPLUS;} /*インクリメント*/
"=="                        { return EQ;}   /*等しい*/
">="                        { return GE;}   
"<="                        { return LE;}
{alpha}({alpha}|{digit})*   { return IDENT; } /*識別子*/
{digit}+                    { return NUM; } /*整数*/
[*%/+\-=();{}<>:]               { return yytext[0]; } /*演算子と括弧*/
{white}                     { ; } /*空白文字は無視*/

[Lex] flex generates lex.yy.c successfully

In [66]:
/* yacc f3.yacc */
%union {
  Node* np;
  int i;
  char* s;
}
%type <s> var
%type <np> stlist stat expr prim cond case_e caselist_e /*型の指定*/
%token NUM /*数字*/
%token IDENT /*識別子*/
%token READ /*文字入力*/
%token PRINT /*文字出力*/
%token WHILE /*while文*/
%token PLUSPLUS /*インクリメント*/
%token EQ /*等しい*/
%token LE /*小なりイコール*/
%token GE /*大なりイコール*/
%token SWITCH /*switch文*/
%token CASE /*case文*/
%token DEFAULT /*default文*/
%token IF /*if文*/
%token ELSE /*else文*/


%%
prog   : IDENT '{' stlist '}'              { emit_c($3); return 0; }
       ;

case_e : CASE expr ':' stat                { $$ = createNode(T_CASE, $2, $4); }
       | DEFAULT ':' stat                  { $$ = createNode(T_DEFAULT, NULL, $3); }
       ;

caselist_e
       : case_e                            { $$ = $1; }
       | caselist_e case_e                 { $$ = createNode(T_STLIST, $1, $2); }
       ;

/*複数命令の定義*/
stlist :                                   { $$ = NULL; }
       | stlist stat                       { $$ = createNode(T_STLIST, $1, $2); }
       ;

/*命令の定義*/
stat   : var '=' expr ';'                  { $$ = createNode(T_ASSIGN, $1, $3); }
       | READ var ';'                      { $$ = createNode(T_READ, $2, NULL); }
       | PRINT expr ';'                    { $$ = createNode(T_PRINT, $2, NULL); }
       | WHILE '(' cond ')' stat           { $$ = createNode(T_WHILE, $3, $5); }
       | WHILE '(' cond ')' '{' stlist '}' { $$ = createNode(T_WHILE, $3, $6); }
       | SWITCH '(' expr ')' '{' caselist_e '}' { $$ = createNode(T_SWITCH, $3, $6); }
       | IF '(' cond ')' '{' stlist '}' ELSE '{' stlist '}'{Node* pair = createNode(T_STMT_PAIR, $6, $10);$$ = createNode(T_IFELSE, $3, pair);}
       | IF '(' cond ')' stat { $$ = createNode(T_IF, $3, $5); }
       ;
/*数式の定義*/
expr   : prim                              { $$ = $1; }
       | expr '+' prim                     { $$ = createNode(T_ADD, $1, $3); }
       | expr '-' prim                     { $$ = createNode(T_SUB, $1, $3); }
       | expr '*' prim                     { $$ = createNode(T_MUL, $1, $3); }
       | expr '/' prim                     { $$ = createNode(T_DIV, $1, $3); }
       | expr '%' prim                     { $$ = createNode(T_MOD, $1, $3); }
       ;

prim   : NUM                               { $$ = createNode(T_NUM, atoi(yytext), NULL); }
       | var                               { $$ = createNode(T_VAR, $1, NULL); }
       | '(' expr ')'                      { $$ = $2; }
       | var PLUSPLUS                      { $$ = createNode(T_PP, $1, NULL); }
       ;

/*変数の定義*/
var    : IDENT                             { $$ = lookup(yytext); }
       ;
/*比較演算子の定義*/
cond   : expr '<' expr                     { $$ = createNode(T_LT, $1, $3); }
       | expr '>' expr                     { $$ = createNode(T_GT, $1, $3); }
       | expr GE expr                      { $$ = createNode(T_GE, $1, $3); }
       | expr LE expr                      { $$ = createNode(T_LE, $1, $3); }
       | expr EQ expr                      { $$ = createNode(T_EQ, $1, $3); }
       ;
%%


[Yacc] bison generates y.tab.c successfully

In [125]:
/* c f3.c */
#include <stdio.h>
#include <stdlib.h>
#include <string.h>

struct stab {
  int val;
  char name[20];
} stab[100];
int stabuse = 0;

extern char *yytext;

#define T_STLIST    1
#define T_ASSIGN    2
#define T_READ      3
#define T_PRINT     4
#define T_ADD       5
#define T_SUB       6
#define T_NUM       7
#define T_VAR       8
#define T_LIST_VAR  9
#define T_WHILE     10
#define T_LT        11
#define T_GT        12
#define T_LE        13
#define T_GE        14
#define T_EQ        15
#define T_PP        16
#define T_CASE      17
#define T_SWITCH    18
#define T_DEFAULT   19
#define T_IF        20
#define T_IFELSE    21
#define T_STMT_PAIR 22
#define T_MUL       23
#define T_DIV       24
#define T_MOD       25

typedef struct Node {
    int node_type;
    struct Node* left;
    struct Node* right;
} Node;

/* 文字列 name[index] を生成 */
char* make_indexed_name(const char* name, int index) {
    char buf[256];
    snprintf(buf, sizeof(buf), "%s[%d]", name, index);
    return strdup(buf);
}

/* シンボルテーブル: s があれば位置を返し、なければ追加して位置を返す */
int lookup(char *s) {
  int i;
  for (i = 0; i < stabuse; ++i) {
    if (strcmp(stab[i].name, s) == 0) {
      return i;
    }
  }
  if (stabuse >= 99) {
    printf("table overflow,\n");
    exit(1);
  }
  strcpy(stab[stabuse].name, s);
  return stabuse++;
}

/* 新しいノードを作る関数 */
Node* createNode(int node_type, Node* left, Node* right) {
    Node* newNode = (Node*)malloc(sizeof(Node));
    newNode->node_type = node_type;
    newNode->left = left;
    newNode->right = right;
    return newNode;
}


void _emit_c(Node* node) {
  if (node == NULL) {
    return;
  }
  switch (node->node_type) {
    case T_STLIST:
      if (node->left != NULL) { _emit_c(node->left); }
      _emit_c(node->right);
      break;
    /*while文の実装*/
    case T_WHILE:
        printf("\twhile (");
        _emit_c(node->left); 
        printf(") {\n");
        _emit_c(node->right); 
        printf("\t}\n");
        break;
    /*=による割り当ての実装*/
    case T_ASSIGN:
      printf("\tv%d = ", node->left);
      _emit_c(node->right);
      printf(";\n");
      break;
    /*readならscanfで文字入力*/
    case T_READ:
      printf("\tscanf(\"%%d\", &v%d);\n", node->left);
      break;
    /*printの実装*/
    case T_PRINT:
      printf("\tprintf(\"%%d\",");
      _emit_c(node->left);
      printf(");\n");
      break;
    /*加算の実装*/
    case T_ADD:
      _emit_c(node->left);
      printf(" + ");
      _emit_c(node->right);
      break;
    /*引き算の実装*/
    case T_SUB:
      _emit_c(node->left);
      printf(" - ");
      _emit_c(node->right);
      break;
    /*掛け算の実装*/
    case T_MUL:
      _emit_c(node->left);
      printf(" * ");
      _emit_c(node->right);
      break;
    /*割り算の実装*/
    case T_DIV:
      _emit_c(node->left);
      printf("/");
      _emit_c(node->right);
      break;
    /*余り演算の実装*/
    case T_MOD:
      _emit_c(node->left); 
      printf(" %% ");
      _emit_c(node->right);
      break;
    /*数値ならば値をprintするだけ*/
    case T_NUM:
      printf("%d", node->left);
      break;
    /*変数ならばv+変数番号を出力*/
    case T_VAR:
      printf("v%d", node->left);
      break;
    case T_LT:
      _emit_c(node->left);
      printf("<");
      _emit_c(node->right);
      break;
    /*大なりの実装*/
    case T_GT:
      _emit_c(node->left);
      printf(">");
      _emit_c(node->right);
      break;
    /*小なりイコールの実装*/
    case T_LE:
      _emit_c(node->left);
      printf("<=");
      _emit_c(node->right);
      break;
    /*大なりイコールの実装*/
    case T_GE:
      _emit_c(node->left);
      printf(">=");
      _emit_c(node->right);
      break;
    /*等しいの実装*/
    case T_EQ:
      _emit_c(node->left);
       printf("==");
      _emit_c(node->right);
      break;
    case T_PP:
      printf("v%d++", node->left);
      break;

    /*SWITCH文の定義*/
    case T_SWITCH: {
        printf("\tswitch (");
        _emit_c(node->left);     /* 判定式 */
        printf(") {\n");
        _emit_c(node->right);    /* case / default 連結リスト */
        printf("\t}\n");
        break;
    }
    /*各caseの出力*/
    case T_CASE:
        printf("\tcase ");
        _emit_c(node->left);           /* ラベル値 */
        printf(":\n");
        _emit_c(node->right);          /* 本体 */
        printf("\t\tbreak;\n");
        break;
    /*defaultの出力*/
    case T_DEFAULT:
        printf("\tdefault:\n");
        _emit_c(node->right);          /* 本体 */
        printf("\t\tbreak;\n");
        break;
    case T_IF:
      printf("\tif (");
      _emit_c(node->left);   // 条件式
      printf(") {\n");
      _emit_c(node->right);  // then 節
      printf("\t}\n");
      break;
    case T_IFELSE: {
      printf("\tif (");
      _emit_c(node->left);   // 条件式
      printf(") {\n");
      Node* pair = node->right;
      _emit_c(pair->left);   // then 節
      printf("\t} else {\n");
      _emit_c(pair->right);  // else 節
      printf("\t}\n");
      break;
    }
  }
}

void emit_c(Node* node) {
  printf("#include <stdio.h>\n");
  printf("int main() {\n");
  int i;
  for (i = 0; i < stabuse; i++) {
    printf("\tint v%d;\n", i);
  }
  _emit_c(node);
  printf("}\n");
}

#include "y.tab.c"
#include "lex.yy.c"

int main() {
  yyparse();
  return 0;
}



[C] gcc generates a.out successfully

下記は1~100のうち、３の倍数を表示するコードです

In [126]:
/* a.out */
main {
  x = 1;
  while(x < 100){
  if(x%3==0){
    print x;
  }else{
  }
  x = x + 1;
  }
}

#include <stdio.h>
int main() {
	int v0;
	v0 = 1;
	while (v0<100) {
	if (v0 % 3==0) {
	printf("%d",v0);
	} else {
	}
	v0 = v0 + 1;
	}
}


下記は5の階乗を表示するコードです。

In [127]:
/* a.out */
main {
  x = 5;
  res = 1;
  while (x > 1) {
    res = res * x;
    x = x - 1;
  }
  print res;  
}


#include <stdio.h>
int main() {
	int v0;
	int v1;
	v0 = 5;
	v1 = 1;
	while (v0>1) {
	v1 = v1 * v0;
	v0 = v0 - 1;
	}
	printf("%d",v1);
}


In [128]:
/* a.out */
main {
  x = 5;
  switch (x % 3) {
    case 0: print 3;
    case 1: print 4;
    case 2: print 5;
    default: print 99;
  }
}

#include <stdio.h>
int main() {
	int v0;
	v0 = 5;
	switch (v0 % 3) {
	case 0:
	printf("%d",3);
		break;
	case 1:
	printf("%d",4);
		break;
	case 2:
	printf("%d",5);
		break;
	default:
	printf("%d",99);
		break;
	}
}


# 発展問題1 - コード生成（第3回の内容から）

第3回で，C言語とx64アセンブリ言語に出力するコンパイラを作成しました．それを参考にして，他のプログラミング言語に出力するコンパイラを作成してください．どんなプログラミング言語に出力するかと，必要に応じて実行モデルについてどうしたか（翻訳するときにどう決めたか）をMarkdownにまとめて書いて，コードには適宜コメントを入れてください．

- 自分の知っているどんな言語でもいいです．
  - 言語処理系論で出てきた WebAssemblyについて自分で調べてみて出力できると，おもしろいかもしれない（その場合は呼び出すJavaScriptも書いてほしい）．

どんな言語に出力したのか，ここに書く

In [ ]:
/* lex f4.lex */

In [ ]:
/* yacc f4.yacc */

In [ ]:
/* c f4.c */

In [ ]:
/* a.out */

# 発展問題2 - プログラミングではない用途（第3回の内容から）

第3回で，簡単な英文解析プログラムを作成しました．同様にして，プログラミング言語ではない用途でLex・Yaccを使用したものを自由に作成して，以下にコードを載せてください．どんなプログラムかをMarkdownでまとめて書いて，コードには適宜コメントを入れてください．

- 英文の例を単純に拡張するのでもよいです．
- JSONやXMLのようなデータを読み込むプログラムでもよいです．
- コマンドのオプションを読み込むときに使うプログラムでもよいです．
  - たとえば，`ssh`だったら（`% ssh -l ユーザー名`）のようにコマンドを書くけれど，それを解釈するプログラム．間違っていたら`man`コマンドの結果のように使い方を表示すしてみるなど

どんな機能を追加したのか，ここに書く

In [ ]:
/* lex f5.lex */

In [ ]:
/* yacc f5.yacc */

In [ ]:
/* c f5.c */

In [ ]:
/* a.out */

# 発展問題3 - 最適化

授業資料で作成したコンパイラは，抽象構文木を作成しただけでした．
抽象構文木の構造をいじってやれば，コンパイラに最適化を実装できます．この課題では，自由に考えて（調べて）自分が作成したコンパイラになんらかの最適化手法を取り入れてください．

- ここまでの問題のどのバージョンのコンパイラでもよいです．
  - 最適化の効果が見やすいので，C言語にコード生成してくれるとありがたい． 
- たとえば，
  - 別々の式になっていた足し算を1つにマージするとか（実際問題として実行速度が上がるわけではないが，抽象構文木の形を変える練習として）
  - [SSA](https://ja.wikipedia.org/wiki/%E9%9D%99%E7%9A%84%E5%8D%98%E4%B8%80%E4%BB%A3%E5%85%A5) 形式に変換してみるとか
  - ループ最適化とか

In [1]:
/* lex f6.lex */
alpha   [a-zA-Z]
digit   [0-9]
white   [\n\t ]
%%
read                        { return READ; }
print                       { return PRINT; }
{alpha}({alpha}|{digit})*   { return IDENT; }
{digit}+                    { return NUM; }
[+\-=();{}]                 { return yytext[0]; }
{white}                     { ; }

[Lex] flex generates lex.yy.c successfully

In [2]:
/* yacc f6.yacc */
%union {
  Node* np;
  int i;
}
%type <np> stlist stat expr prim var
%token NUM;
%token IDENT;
%token READ;
%token PRINT;
%%
prog : IDENT '{' stlist '}'
      {
          traverse_tree(opt($3,NULL,NULL));   /* AST 変換 */
          return 0;
      };
stlist :                            { $$ = NULL; }
       | stlist stat                { $$ = createNode(T_STLIST, $1, $2); }
       ;
stat   : var '=' expr ';'           { $$ = createNode(T_ASSIGN, $1, $3); }
       | READ var ';'               { $$ = createNode(T_READ, $2, NULL); }
       | PRINT expr ';'             { $$ = createNode(T_PRINT, $2, NULL); }
       ;
expr   : prim              { $$ = $1; }
       | expr '+' prim     { $$ = createNode(T_ADD, $1, $3); }
       | expr '-' prim     { $$ = createNode(T_SUB, $1, $3); }
       ;
prim :  NUM    { $$ = createNode(T_NUM, INT_PTR(atoi(yytext)), NULL); }
       | var  { $$ = $1; }  
       | '(' expr ')'      { $$ = $2; }
       ;
var    : IDENT { $$ = createNode(T_VAR, INT_PTR(lookup(yytext)), NULL); }
       ;

[Yacc] bison generates y.tab.c successfully

In [240]:
/* c f6.c */
#include <stdio.h>
struct stab {
  int val;
  char name[20];
} stab[100];
int stabuse = 0;

extern char *yytext;
#include <stdint.h> 
#define INT_PTR(i) ((Node*)(intptr_t)(i))
#define PTR_INT(p) ((int)(intptr_t)(p))

#define T_STLIST 1
#define T_ASSIGN 2
#define T_READ   3
#define T_PRINT  4
#define T_ADD    5
#define T_SUB    6
#define T_NUM    7
#define T_VAR    8

typedef struct Node {
    int node_type;
    struct Node* left;
    struct Node* right;
} Node;


// 引数にとる文字列が表にあれば見つかった位置を返して，なければ追加する関数
int lookup(char *s) {
  int i;
  // 表の使っているところを順に見ていって
  for (i = 0; i < stabuse; ++i) {
    if (strcmp(stab[i].name, s) == 0) {
      // 見つかった場合はその位置を返す
      return i;
    }
  }
  if (stabuse >= 99) {
    // 変数の上限の個数を超えてしまった
    printf("table overflow,\n");
    exit(1);
  }

  // 表になかったので追加する
  strcpy(stab[stabuse].name, s);
  return stabuse++;  // 追加した位置を返す
}

// 新しいノードを作る関数
Node* createNode(int node_type, Node* left, Node* right) {
    Node* newNode = (Node*)malloc(sizeof(Node));
    newNode->node_type = node_type;
    newNode->left = left;
    newNode->right = right;
    return newNode;
}

// ASTに変更を加える関数 (例: T_ADDノードの左右の子を交換する)
Node* opt(Node* node,Node* parent, char lr) {
  if (node == NULL) {
    printf("node is NULL");
    return NULL;
  }
  // 現在のノードに対する処理
  switch (node->node_type) {
    case T_STLIST:
      if (node->left != NULL) { opt(node->left,node,"l");}
      opt(node->right,node,'r');
      break;
    case T_ASSIGN:
      opt(node->right,node,'r');
      break;
    case T_PRINT:
      opt(node->left,node,'l');
      break;
    case T_ADD:
      if (node->left->node_type == T_NUM && node->right->node_type == T_NUM) {
         Node* newNode = createNode(T_NUM, INT_PTR(PTR_INT(node->left->left) + PTR_INT(node->right->left)), NULL);
         if (lr == 'r') parent->right = newNode;
         else parent->left  = newNode;
      }else if(node->left->node_type == T_NUM && PTR_INT(node->left->left) == 0){
        if(lr == 'r') parent->right = node->right;
        else parent->left  = node->right;
      }else if(node->right->node_type == T_NUM && PTR_INT(node->right->left) == 0){
        if(lr == 'r') parent->right = node->left;
        else parent->left  = node->left;
      }else{
        opt(node->left,node,'l');
        opt(node->right,node,'r');
      }
    break;

      break;
  }
  return node; // 変更後の(あるいは変更しない)ノードを返す
}

// ノードをたどる関数
void traverse_tree(Node* node) {
  if (node == NULL) {
    printf("node is NULL\n");
    return;
  }
  switch (node->node_type) {
    case T_STLIST:
      if (node->left != NULL) { traverse_tree(node->left); printf("; "); }
      traverse_tree(node->right);
      break;
    case T_READ:
      printf("read v%d\n", node->left);
      break;
    case T_PRINT:
      printf("print ");
      traverse_tree(node->left);
      printf("\n");
      break;
    case T_ADD:
      printf("(");
      traverse_tree(node->left);
      printf(" + ");
      traverse_tree(node->right);
      printf(")");
      break;
    case T_SUB:
      printf("(");
      traverse_tree(node->left);
      printf(" - ");
      traverse_tree(node->right);
      printf(")");
      break;
    case T_NUM:
      printf("%d", PTR_INT(node->left));
      break;
    case T_VAR:
      printf("v%d", PTR_INT(node->left));
      break;
    case T_ASSIGN:
       /* the left child is a VAR node – print it properly */
      traverse_tree(node->left);
      printf(" = ");
      traverse_tree(node->right);
      printf("\n");
      break;

  }
}

#include "y.tab.c"
#include "lex.yy.c"

int main() {
  yyparse();
  return 0;
}

[C] gcc generates a.out successfully

In [3]:
/* a.out */
main {
  x = 5 + 5;
  print x;
  y = x + 0;
  print y;
}

v0 = 10
; print v0
; v1 = v0
; print v1
